In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
path_to_csv = './data/'
csvs = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]
#csvs = ['knitting.csv']
csv_dict = dict()
for page in csvs:
    #url = "https://raw.githubusercontent.com/rer145/cis572-project/master/data/"+str(page)+".csv"
    #raw_data = requests.get(url).content
    #df = pd.read_csv(io.StringIO(raw_data.decode('utf-8')))
    df = pd.read_csv(path_to_csv + page)
    csv_dict[page] = df

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# remove columns that aren't predictive
# remove columns that are ngrams
for key in csv_dict.keys():
    csv_dict[key].drop('post_gilded', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_silver', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_gold', axis=1, inplace=True)
    csv_dict[key].drop('post_gilded_platinum', axis=1, inplace=True)
    csv_dict[key].drop('post_likes', axis=1, inplace=True)
    csv_dict[key].drop('post_num_comments', axis=1, inplace=True)
    csv_dict[key].drop('post_num_crossposts', axis=1, inplace=True)
    csv_dict[key].drop('post_num_reports', axis=1, inplace=True)
    csv_dict[key].drop('post_ups', axis=1, inplace=True)
    csv_dict[key].drop('post_downs', axis=1, inplace=True)
    
    cols = [col for col in csv_dict[key].columns if '*' in col]
    csv_dict[key].drop(cols, axis=1, inplace=True)

In [4]:
csv_dict['knitting.csv'].columns

Index(['post_score', 'post_archived', 'post_distinguished', 'post_edited',
       'post_is_original_content', 'post_is_video', 'post_over_18',
       'post_pinned', 'subreddit', 'subreddit_subscribers',
       'posted_len_minutes', 'post_title_sentiment', 'post_title_characters',
       'post_title_words', 'post_title_uniq_words', 'post_title_stop_words',
       'post_title_non_stop_words', 'post_text_sentiment',
       'post_text_characters', 'post_text_words', 'post_text_uniq_words',
       'post_text_stop_words', 'post_text_non_stop_words', 'day_of_week'],
      dtype='object')

In [5]:
#Remove columns that only have a singular value
for key in csv_dict.keys():
    unique = [c for c in csv_dict[key].columns if len(set(csv_dict[key][c])) == 1]
    print(key)
    print(unique)
    print()
    csv_dict[key].drop(unique, axis=1, inplace=True)

askmen.csv
['post_archived', 'post_is_original_content', 'post_is_video', 'post_pinned', 'subreddit', 'subreddit_subscribers', 'day_of_week']

askwomen.csv
['post_archived', 'post_is_original_content', 'post_is_video', 'post_pinned', 'subreddit', 'day_of_week']

aww-metadata.csv
['post_archived', 'post_edited', 'post_over_18', 'post_pinned', 'subreddit', 'posted_len_minutes']

aww-no-metadata.csv
['post_archived', 'post_edited', 'post_over_18', 'post_pinned', 'subreddit', 'posted_len_minutes', 'post_text_sentiment', 'post_text_characters', 'post_text_words', 'post_text_uniq_words', 'post_text_stop_words', 'post_text_non_stop_words', 'day_of_week']

aww.csv
['post_archived', 'post_edited', 'post_over_18', 'post_pinned', 'subreddit', 'posted_len_minutes', 'post_text_sentiment', 'post_text_characters', 'post_text_words', 'post_text_uniq_words', 'post_text_stop_words', 'post_text_non_stop_words', 'day_of_week']

conspiracy.csv
['post_archived', 'post_is_original_content', 'post_is_video', 

In [6]:
for key in csv_dict.keys():
    print('parsing', key)
    #Change post_distinguished to numerical values, only None and Moderator values
    if 'post_distinguished' in csv_dict[key]:
        csv_dict[key]['post_distinguished'] = csv_dict[key]['post_distinguished'].map({None: 0, 'moderator': 1})
        
    #Mark edited posts with a 1
    if 'post_edited' in csv_dict[key]:
        csv_dict[key]['post_edited'] = (csv_dict[key]['post_edited'] == 'False')*1
        
    #Make post_over_18 binary
    if 'post_over_18' in csv_dict[key]:
        csv_dict[key]['post_over_18'] = csv_dict[key]['post_over_18'].astype('category').cat.codes
        
    #csv_dict[key] = csv_dict[key].dropna(csv_dict[key].median())

print('done')

parsing askmen.csv
parsing askwomen.csv
parsing aww-metadata.csv
parsing aww-no-metadata.csv
parsing aww.csv
parsing conspiracy.csv
parsing fitness.csv
parsing knitting.csv
done


In [7]:
for key in csv_dict.keys():
    csv_dict[key] = csv_dict[key].sort_values('post_score', ascending=False)
    cutoff_pos = round(csv_dict[key].shape[0]*.25)
    cutoff_score = csv_dict[key].iloc[cutoff_pos]['post_score']
    print('Subreddit:',key,'\t Number of popular posts:',cutoff_pos,'\t Popular score cutoff:',int(cutoff_score))
    
    csv_dict[key].loc[csv_dict[key]['post_score'] >= cutoff_score, 'popular'] = 1
    csv_dict[key].loc[csv_dict[key]['post_score'] < cutoff_score, 'popular'] = 0

Subreddit: askmen.csv 	 Number of popular posts: 248 	 Popular score cutoff: 19
Subreddit: askwomen.csv 	 Number of popular posts: 248 	 Popular score cutoff: 21
Subreddit: aww-metadata.csv 	 Number of popular posts: 247 	 Popular score cutoff: 36
Subreddit: aww-no-metadata.csv 	 Number of popular posts: 247 	 Popular score cutoff: 36
Subreddit: aww.csv 	 Number of popular posts: 247 	 Popular score cutoff: 36
Subreddit: conspiracy.csv 	 Number of popular posts: 247 	 Popular score cutoff: 39
Subreddit: fitness.csv 	 Number of popular posts: 239 	 Popular score cutoff: 14
Subreddit: knitting.csv 	 Number of popular posts: 249 	 Popular score cutoff: 112


In [8]:
def prepare_data(data):
    #remove cols
    if 'post_score' in data.columns:
        data.drop('post_score', axis=1, inplace=True)
    
    #split into test and training
    X = data.drop('popular', axis=1)
    y = data['popular']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
    
    #split into validation and training
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=50)
    
    #scale the data
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    #print('Data sizes')
    #print('X_train', X_train.shape)
    #print('X_val', X_val.shape)
    #print('X_test', X_test.shape)
    #print('y_train', y_train.shape)
    #print('y_val', y_val.shape)
    #print('y_test', y_test.shape)
    
    #return data
    return X_train, X_val, X_test, y_train, y_val, y_test

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gnb = GaussianNB()
knn = KNeighborsClassifier()
svc = SVC()

classifiers = [dtc, rfc, gnb, knn, svc]

In [10]:
names = ['Decision Tree', 'Random Forest', 'Gaussian Naive Bayes', 'K-Nearest Neighbors', 'Support Vector Classifer']
cols = ['# Instances', '# Features'] + names + ['Hard Voting', 'Soft Voting', 'Bagging']
val_results = pd.DataFrame(index=csv_dict.keys(), columns=cols)
val_results.head()

,# Instances,# Features,Decision Tree,Random Forest,Gaussian Naive Bayes,K-Nearest Neighbors,Support Vector Classifer,Hard Voting,Soft Voting,Bagging
askmen.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
askwomen.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aww-metadata.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aww-no-metadata.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aww.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for key in csv_dict.keys():
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(csv_dict[key])
    print('Subreddit:', key)
    for name, algo in zip(names, classifiers):
        algo.fit(X_train, y_train)
        pred = algo.predict(X_val)
        
        val_results.loc[key]['# Instances'] = X_train.shape[0]
        val_results.loc[key]['# Features'] = X_train.shape[1]
        val_results.loc[key][name] = round(accuracy_score(y_val, pred), 3)
print('done')

NameError: name 'train_test_split' is not defined